<a href="https://colab.research.google.com/github/ReanSchwarzer1/Asia-Uni-Final-Group2-Project/blob/main/resnet_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Asia University Winter Program '21 Group 2 Project

## Complimentary notebook to try to use ResNet architecture

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/age-gender-and-ethnicity-face-data-csv/age_gender.csv


In [ ]:
data = pd.read_csv('/kaggle/input/age-gender-and-ethnicity-face-data-csv/age_gender.csv')
data['pixels'] = data.pixels.apply(lambda x: x.split(' '))
data['pixels'] = data.pixels.apply(lambda x: np.array([int(v) for v in x]))
data['pixels'] = data.pixels.apply(lambda x: x.reshape(48,48))

In [ ]:
import os # accessing directory structure
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt # plotting
%matplotlib inline

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from  IPython.display import display
import plotly.express as px

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, experimental, MaxPool2D, BatchNormalization
from tensorflow.keras.losses import sparse_categorical_crossentropy, binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau 
from tensorflow.data import Dataset
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.random import set_seed
from tensorflow.keras.utils import to_categorical
from tensorflow import test
import random
import keras

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(data.drop(['age','ethnicity','gender','img_name'], axis=1),
                                                  data[['age','ethnicity','gender']], random_state=0, test_size=0.25)


def preprocess (df, y):
    """Redim df"""
    X = np.zeros((len(df.values), 48, 48, 1))
    for idx,array in enumerate(df[y]):
        X[idx, :, :, 0] = array
    return X

# We expand dimension to fit with the CNN inputs
Xtrain = preprocess(X_train, 'pixels')
Xval = preprocess(X_val, 'pixels')

# We decided to make prediction only on age but it can easily be done on the other 
ytrain = y_train.age.values
yval = y_val.age.values

## SE-RESNET Block

We are implementing a residual bloc and an SE block combined together which can make really good predictions.

A SE block is not looking for spatial patterns like CNN, it learns the caracteristics which work well in group. Like nose and mouth are relatively close on a face the NN will expect to see eyes. If it constats a high activation for the nose and mouth feature cards and a medium one for the eyes, the block will excite the last one.

A block SE has only 3 layers and pulls out a vector which will multiply the feature cards of a previous resnet block.

In [ ]:
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [keras.layers.Conv2D(filters, 3, strides=strides, padding="same",use_bias=False),
                            keras.layers.BatchNormalization(), # Normalize the outputs
                            self.activation,
                            keras.layers.Conv2D(filters, 3, strides=1, padding='same', use_bias=False),
                            keras.layers.BatchNormalization()]
        self.skip_layers = [
            keras.layers.Conv2D(filters, 1, strides=strides,padding="same",use_bias=False),
            keras.layers.BatchNormalization()
        ]
    
    # We don't forget the call method which is called during the training and prediction
    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)
    
class SEBloc(keras.layers.Layer):
    def __init__(self, pool, **kwargs):
        super().__init__(**kwargs)
        self.main_layers = [keras.layers.AveragePooling2D(
                                pool_size=pool, strides=1, padding="same"), # pool_size is important, we need a scalar per feature card
                            keras.layers.Dense(5, activation='relu'), # embedding
                            keras.layers.Dense(64, activation='sigmoid')] # outputs 
    
    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        return Z

## SENET model

In [ ]:
import tensorflow as tf

In [ ]:
EPOCHS = 5
BATCH_SIZE = 32

inputs = tf.keras.Input(shape=(48,48,1), dtype="float32")
x = keras.layers.Conv2D(64, 5, strides=2, input_shape=[48,48,1])(inputs)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation("relu")(x)
x = keras.layers.MaxPool2D(pool_size=2, strides=2, padding='same')(x)

x_res = ResidualUnit(64, strides=1)(x) # RES

x_se = SEBloc(x.shape[1])(x) #SE

x_res_se = keras.layers.Multiply()([x_res, x_se]) # Multiply outputs of SE and RES
x = keras.layers.Add()([x_res_se, x])

x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Flatten()(x)
output = keras.layers.Dense(1, activation='relu')(x) # One output with relu for the regression
model = tf.keras.Model(inputs, output)

## Learning rate

In [ ]:
class ExponentialLearningRate(keras.callbacks.Callback):
    
    def __init__(self, K, factor):
        self.factor = factor
        self.rates = []
        self.losses = []
        self.K = K
        
    def on_batch_end(self, batch, logs):
        
        self.rates.append(self.K.get_value(self.model.optimizer.lr))
        self.losses.append(logs["loss"])
        self.K.set_value(self.model.optimizer.lr, self.model.optimizer.lr * self.factor)
        
        
def bestLearningRate():
        
        print("\n\n********************** Learning rate calculation ******************\n\n")
        K = keras.backend
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[tf.keras.metrics.MeanAbsoluteError()])
        expon_lr = ExponentialLearningRate(K,factor=1.0003)
        model.fit(Xtrain, ytrain, validation_data=(Xval, yval), epochs = 20, callbacks=[expon_lr])
        print("*************************************************************************\n\n")
        
        print("********************** Loss as function of learning rate plot displayed ********************\n\n")
        
        fig = px.line(
        x=expon_lr.rates, y=expon_lr.losses,
        labels={'index': 'learning rate', 'value': 'loss'}, 
        title='Training History')
        fig.show()
        
        id_min = np.argmin(expon_lr.losses)
        return expon_lr.rates[id_min]
        
lr = bestLearningRate()
print('the learning rate is: ',lr)



********************** Learning rate calculation ******************


Epoch 1/20
556/556 [==============================] - 34s 60ms/step - loss: 291.4025 - mean_absolute_error: 12.9176 - val_loss: 306.4839 - val_mean_absolute_error: 14.7766
Epoch 2/20
556/556 [==============================] - 32s 58ms/step - loss: 151.6863 - mean_absolute_error: 9.3068 - val_loss: 220.3962 - val_mean_absolute_error: 10.8263
Epoch 3/20
556/556 [==============================] - 33s 59ms/step - loss: 132.0051 - mean_absolute_error: 8.6252 - val_loss: 287.4597 - val_mean_absolute_error: 14.1021
Epoch 4/20
556/556 [==============================] - 32s 58ms/step - loss: 126.5314 - mean_absolute_error: 8.4083 - val_loss: 426.4958 - val_mean_absolute_error: 17.9101
Epoch 5/20
556/556 [==============================] - 32s 58ms/step - loss: 122.3401 - mean_absolute_error: 8.3072 - val_loss: 668.2391 - val_mean_absolute_error: 22.9683
Epoch 6/20
556/556 [==============================] - 32s 58ms/step - lo

## Training 

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_loss')<91):
            print("\nReached 110 val_loss so cancelling training!")
            self.model.stop_training = True
        
callback = myCallback()
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

SGD = tf.keras.optimizers.Adam(learning_rate=0.0035) 
model.compile(loss='mse', optimizer=SGD ,metrics=[tf.keras.metrics.MeanAbsoluteError()])

history = model.fit(Xtrain, ytrain, batch_size=BATCH_SIZE, epochs=5, validation_data=(Xval, yval), callbacks = [callback, reduce_lr], verbose=1)

## Loss graph

In [ ]:
fig = px.line(
    history.history, y=['loss', 'val_loss'],
    labels={'index': 'epoch', 'value': 'loss'}, 
    title='Training History')
fig.show()